In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import constants as sc

from DeepFMKit import physics
# NOTE: The notebook now only needs to import 'Experiment', not the specific workers
from DeepFMKit.experiments import Experiment 

# --- 1. Define the Physics Factory for the Experiment ---
# This function must be at the top level of the notebook for multiprocessing.
def create_distortion_configs(params: dict) -> dict:
    """
    Takes a dictionary of trial parameters and returns the fully configured
    physics objects needed for the simulation.
    """
    m_main = params['m_main']
    m_witness = params['m_witness']
    distortion_amp = params['distortion_amp']
    distortion_phase = params.get('distortion_phase', 0.0) # Default to 0 if not stochastic

    # Create the base config objects
    laser_config = physics.LaserConfig()
    main_ifo_config = physics.InterferometerConfig()
    
    # Configure the objects based on the input parameters
    opd_main = main_ifo_config.meas_arml - main_ifo_config.ref_arml
    if opd_main == 0: opd_main = 0.2 # Avoid division by zero
    laser_config.df = (m_main * sc.c) / (2 * np.pi * opd_main)
    laser_config.df_2nd_harmonic_frac = distortion_amp
    laser_config.df_2nd_harmonic_phase = distortion_phase

    # Create the witness interferometer configuration
    witness_ifo_config = physics.InterferometerConfig()
    if laser_config.df > 0 and 'm_witness' in params:
        opd_witness = (m_witness * sc.c) / (2 * np.pi * laser_config.df)
        witness_ifo_config.ref_arml = 0.01
        witness_ifo_config.meas_arml = witness_ifo_config.ref_arml + opd_witness
        f0 = sc.c / laser_config.wavelength
        static_fringe_phase = (2 * np.pi * f0 * opd_witness) / sc.c
        witness_ifo_config.phi = (np.pi / 2.0) - static_fringe_phase
    
    return {
        'laser_config': laser_config,
        'main_ifo_config': main_ifo_config,
        'witness_ifo_config': witness_ifo_config
    }

print("=" * 60)
print("Validating W-DFMI Bias Correction using the Experiment Runner")
print("=" * 60)

# 1. Declaratively Define the Experiment
exp = Experiment(description="W-DFMI Bias Correction vs. Modulation Non-Linearity")

exp.add_axis('distortion_amp', np.linspace(0.00, 0.1, 11))
exp.set_static({'m_main': 20.0, 'm_witness': 0.04})

exp.n_trials = 50
exp.add_stochastic_variable('distortion_phase', lambda: np.random.uniform(0, 2 * np.pi))

exp.set_config_factory(create_distortion_configs)

exp.add_analysis(name='wdfmi_fit', fitter_method='wdfmi_ortho')
exp.add_analysis(name='nls_fit', fitter_method='nls', fitter_kwargs={'ndata': 30, 'parallel': False})

# 2. Run the Experiment
results = exp.run()

In [ ]:
# Calculate the bias for each analysis after the fact
m_true = exp.static_params['m_main']
nls_bias = results['nls_fit']['m']['mean'] - m_true
wdfmi_bias = results['wdfmi_fit']['m']['mean'] - m_true

nls_std = results['nls_fit']['m']['std']
wdfmi_std = results['wdfmi_fit']['m']['std']

fig, ax = plt.subplots(figsize=(12, 7))

ax.errorbar(
    results['axes']['distortion_amp'] * 100, 
    nls_bias, 
    yerr=nls_std,
    fmt='o-', capsize=5, color='tab:red', label='Standard NLS Fitter Bias'
)
ax.errorbar(
    results['axes']['distortion_amp'] * 100, 
    wdfmi_bias, 
    yerr=wdfmi_std,
    fmt='s-', capsize=5, color='tab:green', label='W-DFMI Fitter Bias (Corrected)'
)

ax.axhline(0, color='k', linestyle='--', linewidth=1, alpha=0.7)
ax.set_xlabel('2nd Harmonic Distortion Amplitude (%)', fontsize=14)
ax.set_ylabel(r"Bias in Modulation Depth, $\delta m = \hat{m} - m_{\rm true}$ (rad)", fontsize=14)
ax.set_title(exp.description, fontsize=16)
ax.grid(True, which='both', linestyle=':')
ax.legend(fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
from DeepFMKit import physics
from scipy import constants as sc
import numpy as np

def create_distortion_configs(params: dict) -> dict:
    """
    Takes a dictionary of trial parameters and returns the fully configured
    physics objects needed for the simulation.
    """
    m_main = params['m_main']
    m_witness = params['m_witness']
    distortion_amp = params['distortion_amp']
    distortion_phase = params.get('distortion_phase', 0.0)

    laser_config = physics.LaserConfig()
    main_ifo_config = physics.InterferometerConfig()
    
    opd_main = main_ifo_config.meas_arml - main_ifo_config.ref_arml
    if opd_main == 0: opd_main = 0.2
    laser_config.df = (m_main * sc.c) / (2 * np.pi * opd_main)
    laser_config.df_2nd_harmonic_frac = distortion_amp
    laser_config.df_2nd_harmonic_phase = distortion_phase

    witness_ifo_config = physics.InterferometerConfig()
    if laser_config.df > 0 and 'm_witness' in params:
        opd_witness = (m_witness * sc.c) / (2 * np.pi * laser_config.df)
        witness_ifo_config.ref_arml = 0.01
        witness_ifo_config.meas_arml = witness_ifo_config.ref_arml + opd_witness
        f0 = sc.c / laser_config.wavelength
        static_fringe_phase = (2 * np.pi * f0 * opd_witness) / sc.c
        witness_ifo_config.phi = (np.pi / 2.0) - static_fringe_phase
    
    return {
        'laser_config': laser_config,
        'main_ifo_config': main_ifo_config,
        'witness_ifo_config': witness_ifo_config
    }

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.gridspec import GridSpec

# Import the necessary toolkit components
from DeepFMKit.experiments import Experiment 

# --- 1. Declaratively Define the Experiment ---
exp = Experiment(description="Systematic Bias from Modulation Non-Linearity")

# Define the 2D parameter grid
exp.add_axis('m_main', np.linspace(2, 25, 16))
exp.add_axis('distortion_amp', np.linspace(0, 0.2, 16))

# Define static parameters
exp.set_static({'m_witness': 0.5})

# Define the Monte Carlo simulation
exp.n_trials = 50
exp.add_stochastic_variable('distortion_phase', lambda: np.random.uniform(0, 2 * np.pi))

# Set the function that creates the physics configs for each trial
exp.set_config_factory(create_distortion_configs)

# Define the two analyses to run on the *same* simulated data
exp.add_analysis(name='wdfmi_fit', fitter_method='wdfmi_ortho')
exp.add_analysis(name='nls_fit', fitter_method='nls', fitter_kwargs={'parallel': False})

# --- 2. Run the Experiment ---
# The if __name__ == '__main__': guard is essential for multiprocessing
if __name__ == '__main__':
    results = exp.run()

In [ ]:
# Extract the axes for plotting and analysis
m_range = results['axes']['m_main']
distortion_range = results['axes']['distortion_amp']

# --- THE FIX IS HERE ---
# Instead of using the pre-computed stats, we calculate the bias from the raw trial data.

# Get the full grid of m_fit values for every trial
nls_m_all_trials = results['nls_fit']['m']['all_trials']
wdfmi_m_all_trials = results['wdfmi_fit']['m']['all_trials']

# The m_range (our m_true values) is a 1D array. We need to reshape it
# so it can be broadcast and subtracted from the 3D trial grid.
m_true_grid = m_range[:, np.newaxis, np.newaxis]

# Now, create the grid of BIAS values for every trial
nls_bias_all_trials = nls_m_all_trials - m_true_grid
wdfmi_bias_all_trials = wdfmi_m_all_trials - m_true_grid

# Finally, calculate the statistics on the BIAS grid
nls_bias_mean = np.nanmean(nls_bias_all_trials, axis=-1)
wdfmi_bias_mean = np.nanmean(wdfmi_bias_all_trials, axis=-1)

nls_bias_worst = np.nanmax(np.abs(nls_bias_all_trials), axis=-1)
wdfmi_bias_worst = np.nanmax(np.abs(wdfmi_bias_all_trials), axis=-1)

# --- Plotting ---
fig = plt.figure(figsize=(18, 14))
gs = GridSpec(2, 3, width_ratios=[1, 1, 0.05], wspace=0.3, hspace=0.4)
ax1, ax2 = fig.add_subplot(gs[0, 0]), fig.add_subplot(gs[0, 1])
ax3, ax4 = fig.add_subplot(gs[1, 0]), fig.add_subplot(gs[1, 1])
cax = fig.add_subplot(gs[:, 2])

colors = ["#FFFFFF", "#D6EAF8", "#5499C7", "#154360"]
custom_cmap = LinearSegmentedColormap.from_list("custom_cmap", colors)
vmax = np.pi

def plot_panel(ax, data, title):
    im = ax.pcolormesh(m_range, distortion_range * 100, np.abs(data.T), 
                        cmap=custom_cmap, vmin=0, vmax=vmax, shading='auto')
    ax.set_title(title, fontsize=16, pad=10)
    ax.set_xlabel(r'Modulation Depth, $m$', fontsize=14)
    ax.set_ylabel(r'2nd Harmonic Distortion, $\epsilon$ (%)', fontsize=14)
    return im

plot_panel(ax1, nls_bias_mean, '(a) Standard NLS: Mean Bias')
plot_panel(ax2, wdfmi_bias_mean, '(b) W-DFMI: Mean Bias (Corrected)')
plot_panel(ax3, nls_bias_worst, '(c) Standard NLS: Worst-Case Bias')
im = plot_panel(ax4, wdfmi_bias_worst, '(d) W-DFMI: Worst-Case Bias (Corrected)')

fig.colorbar(im, cax=cax, orientation='vertical', label=r'Absolute Bias, $|\delta m|$ (rad)')
plt.show()